In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Read in data

In [3]:
file_name = '../data/transactions_chunk1.csv'
lines = sc.textFile(file_name)

In [4]:
header = lines.first()
lines = lines.filter(lambda row: row != header)

In [5]:
# Functions for parsing the strings from the CSV file

def toIntSafe(inval):
  try:
    return int(inval)
  except ValueError:
    return None

def toTimeSafe(inval):
  try:
    return datetime.strptime(inval, "%Y-%m-%d")
  except ValueError:
    return None

def toLongSafe(inval):
  try:
    return long(inval)
  except ValueError:
    return None

def toFloatSafe(inval):
  try:
    return float(inval)
  except ValueError:
    return None
    
def stringToPost(row):
  r = row.encode('utf-8').split(',')
  return Row(
    int(r[0]),         # Don't want this column to be nullable
    toLongSafe(r[1]),
    toLongSafe(r[2]),
    toLongSafe(r[3]),
    toLongSafe(r[4]),
    toLongSafe(r[5]),
    toTimeSafe(r[6]),
    toFloatSafe(r[7]),
    r[8],
    toLongSafe(r[9]),
    toFloatSafe(r[10]))  

In [6]:
transactions_schema = StructType([
    StructField("id", LongType(), False),
    StructField("chain", LongType(), True),
    StructField("dept", LongType(), True),
    StructField("category", LongType(), True),
    StructField("company", LongType(), True),
    StructField("brand", LongType(), True),
    StructField("date", DateType(), True),
    StructField("productsize", DoubleType(), True),
    StructField("purchasemeasure", StringType(), True),
    StructField("purchasequantity", LongType(), True),
    StructField("purchaseamount", DoubleType(), True)
    ])

In [7]:
rowRDD = lines.map(lambda p: stringToPost(p))

In [8]:
transactions = sqlContext.createDataFrame(rowRDD, transactions_schema)

# Training data and testing data

In [9]:
train = pd.read_csv('../data/trainHistory.csv')
offers = pd.read_csv('../data/offers.csv')

In [10]:
# Add department to offers
dept = []
for i in range(offers.shape[0]):
    str_category = str(offers['category'].iloc[i])
    if len(str_category) == 4:
        dept.append(str_category[:2])
    else:
        dept.append(str_category[:1])
        
offers['dept'] = dept
offers['dept'] = pd.to_numeric(offers['dept'])

In [11]:
train = pd.merge(train, offers, how='left', on=['offer'])

In [12]:
# Rename columns
train.columns = ['id', 'chain', 'offer', 'market', 'repeattrips', 'repeater', 'offerdate', 
                 'offer_category', 'quantity', 'offer_company', 'offervalue', 'offer_brand',
                 'offer_dept']

In [13]:
# Reencode the target variable
train['repeater'] = np.where(train['repeater'] == 't', '1', '0')
train['repeater'] = pd.to_numeric(train['repeater'])

In [14]:
train['offerdate'] = train['offerdate'].astype(str)

In [15]:
test = pd.read_csv('../data/testHistory.csv')
test = pd.merge(test, offers, how='left', on=['offer'])

In [16]:
# Rename columns
test.columns = ['id', 'chain', 'offer', 'market', 'offerdate', 'offer_category', 'quantity', 
                'offer_company', 'offervalue', 'offer_brand', 'offer_dept']

In [17]:
test['offerdate'] = test['offerdate'].astype(str)

### Add in Shiva's features

In [18]:
train_more = pd.read_csv('../data/train.csv')
train_more.drop(['label', 'offer_value', 'offer_quantity'], 1, inplace=True)

In [19]:
train = train.merge(train_more, on='id', how='left')

In [20]:
test_more = pd.read_csv('../data/test.csv')
test_more.drop(['label', 'offer_value', 'offer_quantity'], 1, inplace=True)

In [21]:
test = test.merge(test_more, on='id', how='left')

### Convert to Spark DataFrame

In [22]:
train_schema = StructType([
    StructField("id", LongType(), False),
    StructField("offer_chain", LongType(), True),
    StructField("offer", LongType(), True),
    StructField("market", LongType(), True),
    StructField("repeattrips", LongType(), True),
    StructField("repeater", IntegerType(), True),
    StructField("offerdate", StringType(), True),
    StructField("offer_category", LongType(), True),
    StructField("quantity", LongType(), True),
    StructField("offer_company", LongType(), True),
    StructField("offervalue", DoubleType(), True),
    StructField("offer_brand", LongType(), True),
    StructField("offer_dept", LongType(), True),
    StructField("total_spend", DoubleType(), True),
    StructField("day_of_week", DoubleType(), True),
    StructField("day_of_month", DoubleType(), True),
    StructField("month", DoubleType(), True),
    StructField("has_bought_company", DoubleType(), True),
    StructField("has_bought_company_q", DoubleType(), True),
    StructField("has_bought_company_a", DoubleType(), True),
    StructField("has_bought_company_30", DoubleType(), True),
    StructField("has_bought_company_q_30", DoubleType(), True),
    StructField("has_bought_company_a_30", DoubleType(), True),
    StructField("has_bought_company_60", DoubleType(), True),
    StructField("has_bought_company_q_60", DoubleType(), True),
    StructField("has_bought_company_a_60", DoubleType(), True),
    StructField("has_bought_company_90", DoubleType(), True),
    StructField("has_bought_company_q_90", DoubleType(), True),
    StructField("has_bought_company_a_90", DoubleType(), True),
    StructField("has_bought_company_180", DoubleType(), True),
    StructField("has_bought_company_q_180", DoubleType(), True),
    StructField("has_bought_company_a_180", DoubleType(), True),
    StructField("has_bought_category", DoubleType(), True),
    StructField("has_bought_category_q", DoubleType(), True),
    StructField("has_bought_category_a", DoubleType(), True),
    StructField("has_bought_category_30", DoubleType(), True),
    StructField("has_bought_category_q_30", DoubleType(), True),
    StructField("has_bought_category_a_30", DoubleType(), True),
    StructField("has_bought_category_60", DoubleType(), True),
    StructField("has_bought_category_q_60", DoubleType(), True),
    StructField("has_bought_category_a_60", DoubleType(), True),
    StructField("has_bought_category_90", DoubleType(), True),
    StructField("has_bought_category_q_90", DoubleType(), True),
    StructField("has_bought_category_a_90", DoubleType(), True),
    StructField("has_bought_category_180", DoubleType(), True),
    StructField("has_bought_category_q_180", DoubleType(), True),
    StructField("has_bought_category_a_180", DoubleType(), True),
    StructField("has_bought_brand", DoubleType(), True),
    StructField("has_bought_brand_q", DoubleType(), True),
    StructField("has_bought_brand_a", DoubleType(), True),
    StructField("has_bought_brand_30", DoubleType(), True),
    StructField("has_bought_brand_q_30", DoubleType(), True),
    StructField("has_bought_brand_a_30", DoubleType(), True),
    StructField("has_bought_brand_60", DoubleType(), True),
    StructField("has_bought_brand_q_60", DoubleType(), True),
    StructField("has_bought_brand_a_60", DoubleType(), True),
    StructField("has_bought_brand_90", DoubleType(), True),
    StructField("has_bought_brand_q_90", DoubleType(), True),
    StructField("has_bought_brand_a_90", DoubleType(), True),
    StructField("has_bought_brand_180", DoubleType(), True),
    StructField("has_bought_brand_q_180", DoubleType(), True),
    StructField("has_bought_brand_a_180", DoubleType(), True),
    StructField("never_bought_company", DoubleType(), True),
    StructField("never_bought_category", DoubleType(), True),
    StructField("never_bought_brand", DoubleType(), True),
    StructField("has_bought_brand_company_category", DoubleType(), True),
    StructField("has_bought_brand_category", DoubleType(), True),
    StructField("has_bought_brand_company", DoubleType(), True)
    ])

In [23]:
train_df = sqlContext.createDataFrame(train, train_schema)

In [24]:
test_schema = StructType([
    StructField("id", LongType(), False),
    StructField("offer_chain", LongType(), True),
    StructField("offer", LongType(), True),
    StructField("market", LongType(), True),
    StructField("offerdate", StringType(), True),
    StructField("offer_category", LongType(), True),
    StructField("quantity", LongType(), True),
    StructField("offer_company", LongType(), True),
    StructField("offervalue", DoubleType(), True),
    StructField("offer_brand", LongType(), True),
    StructField("offer_dept", LongType(), True),
    StructField("total_spend", DoubleType(), True),
    StructField("day_of_week", DoubleType(), True),
    StructField("day_of_month", DoubleType(), True),
    StructField("month", DoubleType(), True),
    StructField("has_bought_company", DoubleType(), True),
    StructField("has_bought_company_q", DoubleType(), True),
    StructField("has_bought_company_a", DoubleType(), True),
    StructField("has_bought_company_30", DoubleType(), True),
    StructField("has_bought_company_q_30", DoubleType(), True),
    StructField("has_bought_company_a_30", DoubleType(), True),
    StructField("has_bought_company_60", DoubleType(), True),
    StructField("has_bought_company_q_60", DoubleType(), True),
    StructField("has_bought_company_a_60", DoubleType(), True),
    StructField("has_bought_company_90", DoubleType(), True),
    StructField("has_bought_company_q_90", DoubleType(), True),
    StructField("has_bought_company_a_90", DoubleType(), True),
    StructField("has_bought_company_180", DoubleType(), True),
    StructField("has_bought_company_q_180", DoubleType(), True),
    StructField("has_bought_company_a_180", DoubleType(), True),
    StructField("has_bought_category", DoubleType(), True),
    StructField("has_bought_category_q", DoubleType(), True),
    StructField("has_bought_category_a", DoubleType(), True),
    StructField("has_bought_category_30", DoubleType(), True),
    StructField("has_bought_category_q_30", DoubleType(), True),
    StructField("has_bought_category_a_30", DoubleType(), True),
    StructField("has_bought_category_60", DoubleType(), True),
    StructField("has_bought_category_q_60", DoubleType(), True),
    StructField("has_bought_category_a_60", DoubleType(), True),
    StructField("has_bought_category_90", DoubleType(), True),
    StructField("has_bought_category_q_90", DoubleType(), True),
    StructField("has_bought_category_a_90", DoubleType(), True),
    StructField("has_bought_category_180", DoubleType(), True),
    StructField("has_bought_category_q_180", DoubleType(), True),
    StructField("has_bought_category_a_180", DoubleType(), True),
    StructField("has_bought_brand", DoubleType(), True),
    StructField("has_bought_brand_q", DoubleType(), True),
    StructField("has_bought_brand_a", DoubleType(), True),
    StructField("has_bought_brand_30", DoubleType(), True),
    StructField("has_bought_brand_q_30", DoubleType(), True),
    StructField("has_bought_brand_a_30", DoubleType(), True),
    StructField("has_bought_brand_60", DoubleType(), True),
    StructField("has_bought_brand_q_60", DoubleType(), True),
    StructField("has_bought_brand_a_60", DoubleType(), True),
    StructField("has_bought_brand_90", DoubleType(), True),
    StructField("has_bought_brand_q_90", DoubleType(), True),
    StructField("has_bought_brand_a_90", DoubleType(), True),
    StructField("has_bought_brand_180", DoubleType(), True),
    StructField("has_bought_brand_q_180", DoubleType(), True),
    StructField("has_bought_brand_a_180", DoubleType(), True),
    StructField("never_bought_company", DoubleType(), True),
    StructField("never_bought_category", DoubleType(), True),
    StructField("never_bought_brand", DoubleType(), True),
    StructField("has_bought_brand_company_category", DoubleType(), True),
    StructField("has_bought_brand_category", DoubleType(), True),
    StructField("has_bought_brand_company", DoubleType(), True)
    ])

In [25]:
test_df = sqlContext.createDataFrame(test, test_schema)

In [26]:
# This function converts the string cell into a date:
stringToDate = F.udf(lambda x: datetime.strptime(x, '%Y-%m-%d'), DateType())

In [27]:
# Convert offerdate to date type
train_df = train_df.withColumn('offerdate', stringToDate(F.col('offerdate')))
test_df = test_df.withColumn('offerdate', stringToDate(F.col('offerdate')))

In [28]:
train_df = train_df.withColumnRenamed('repeater', 'label')

# Feature Engineering

### Get number of refund transactions and total transactions

In [29]:
id_stats = transactions.select('id', 'purchasequantity', 'purchaseamount',
                              F.when(transactions['purchaseamount'] < 0, 1).otherwise(0).alias('return'))\
                      .withColumn('1', F.lit(1))
id_stats = id_stats.groupBy('id').agg(F.sum('1').alias('total_trans'), 
                                      F.sum('return').alias('total_returns'),
                                      F.avg('purchasequantity').alias('avg_purchasequantity'),
                                      F.avg('purchaseamount').alias('avg_purchaseamount'))
id_stats.show()

+--------+-----------+-------------+--------------------+------------------+
|      id|total_trans|total_returns|avg_purchasequantity|avg_purchaseamount|
+--------+-----------+-------------+--------------------+------------------+
|18470775|        350|            8|                1.34|3.1511428571428635|
|14723452|        755|            5|  1.2264900662251657| 4.073933774834406|
|15738658|         39|            0|   1.205128205128205| 4.780000000000001|
|17552659|        591|            4|  1.5245346869712353| 4.931455160744474|
|12996040|        326|            5|  1.2331288343558282|4.7280981595092095|
|16078766|        966|           54|   1.818840579710145| 5.298498964803263|
|18249735|       1557|           60|  1.2594733461785486|  4.13752087347458|
|14989775|        614|           39|   1.231270358306189|3.8022475570032475|
|15073302|        526|           38|  1.3669201520912548| 4.640133079847899|
|16075389|        591|           32|  1.9593908629441625| 7.553604060913664|

In [30]:
# Rename id column of returns table to avoid ambiguous column names
id_stats = id_stats.withColumnRenamed('id', 'returns_id')

In [31]:
train_df = train_df.join(id_stats, train_df['id'] == id_stats['returns_id'], 'left')
test_df = test_df.join(id_stats, test_df['id'] == id_stats['returns_id'], 'left')

In [32]:
train_df = train_df.drop('repeattrips', 'quantity', 'returns_id')
test_df = test_df.drop('quantity', 'returns_id')

In [33]:
# Get average value for imputations
avg_total_trans = id_stats.agg(F.avg(F.col('total_trans'))).first()[0]
avg_total_returns = id_stats.agg(F.avg(F.col('total_returns'))).first()[0]
avg_avg_purchasequantity = id_stats.agg(F.avg(F.col('avg_purchasequantity'))).first()[0]
avg_avg_purchaseamount = id_stats.agg(F.avg(F.col('avg_purchaseamount'))).first()[0]

# Impute missing data
impute_dict = {'total_trans': avg_total_trans, 'total_returns': avg_total_returns,
               'avg_purchasequantity': avg_avg_purchasequantity,
               'avg_purchaseamount': avg_avg_purchaseamount}
train_df = train_df.fillna(impute_dict)
test_df = test_df.fillna(impute_dict)

### Get number of transactions that match the offers

In [35]:
train_trans_agg = train_df.join(transactions, 
                            (train_df.id == transactions.id) & (train_df.offer_chain == transactions.chain), 
                            "left") \
                          .select(train_df["id"], "offer_dept", "dept", "offer_company", "company", 
                                  "offer_brand", "brand", "offer_category", "category") \
                          .withColumn("matchDeptCount", F.expr("case when offer_dept = dept then 1 else 0 end")) \
                          .withColumn("matchCategoryCount", F.expr("case when offer_category = category then 1 else 0 end")) \
                          .withColumn("matchCompanyCount", F.expr("case when offer_company = company then 1 else 0 end")) \
                          .withColumn("matchBrandCount", F.expr("case when offer_brand = brand then 1 else 0 end")) \
                          .groupBy("id") \
                          .agg(F.sum("matchDeptCount").alias("matchDeptCount"),
                               F.sum("matchCategoryCount").alias("matchCategoryCount"),
                               F.sum("matchCompanyCount").alias("matchCompanyCount"),
                               F.sum("matchBrandCount").alias("matchBrandCount"))

In [36]:
train_df = train_df.join(train_trans_agg, train_df.id == train_trans_agg.id, "left").drop(train_trans_agg.id)

In [37]:
test_trans_agg = test_df.join(transactions, 
                            (test_df.id == transactions.id) & (test_df.offer_chain == transactions.chain), 
                            "left") \
                          .select(test_df["id"], "offer_dept", "dept", "offer_company", "company", 
                                  "offer_brand", "brand", "offer_category", "category") \
                          .withColumn("matchDeptCount", F.expr("case when offer_dept = dept then 1 else 0 end")) \
                          .withColumn("matchCategoryCount", F.expr("case when offer_category = category then 1 else 0 end")) \
                          .withColumn("matchCompanyCount", F.expr("case when offer_company = company then 1 else 0 end")) \
                          .withColumn("matchBrandCount", F.expr("case when offer_brand = brand then 1 else 0 end")) \
                          .groupBy("id") \
                          .agg(F.sum("matchDeptCount").alias("matchDeptCount"),
                               F.sum("matchCategoryCount").alias("matchCategoryCount"),
                               F.sum("matchCompanyCount").alias("matchCompanyCount"),
                               F.sum("matchBrandCount").alias("matchBrandCount"))

In [38]:
test_df = test_df.join(test_trans_agg, test_df.id == test_trans_agg.id, "left").drop(test_trans_agg.id)

In [39]:
train_df = train_df.na.fill(0)
test_df = test_df.na.fill(0)

In [68]:
train_df = train_df.drop('offerdate')
test_df = test_df.drop('offerdate')


# Create pipeline

In [40]:
# Get all the offers, categories, brands and companies from both training and test
# Then create a StringIndexer to map categorical values to numeric labels
cols_to_index = ['offer', 'offer_category', 'offer_brand', 'offer_company',
                 'never_bought_company', 'never_bought_category', 'never_bought_brand',
                 'has_bought_brand_company_category', 'has_bought_brand_category', 'has_bought_brand_company']
indexers = []
for col in cols_to_index:
    union_df = train_df.select(col).union(test_df.select(col))
    indexers.append(StringIndexer().setInputCol(col).setOutputCol(col+'idx').fit(union_df))

In [69]:
# One-hot-encoding of categorical variables
encoders = []
for col in cols_to_index:
    encoders.append(OneHotEncoder().setInputCol(col+'idx').setOutputCol(col+'_encoded').setDropLast(False))

In [70]:
# Assemble features
inputCols = [col for col in train_df.columns if col != 'id' or col != 'label']
assembler = VectorAssembler(inputCols=inputCols, outputCol='features')

In [71]:
# Logistic regression
lr = LogisticRegression(maxIter=10, standardization=True)

In [72]:
# Pipeline
stages = indexers + encoders + [assembler, lr]
pipeline = Pipeline(stages=stages)

# Cross-validation

In [73]:
regularization = [0.0001, 0.001]   # for testing purposes
#regularization = [0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
kfolds = 2

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, regularization) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=kfolds)  # use 3+ folds in practice

In [74]:
train_df.cache()
test_df.cache()

DataFrame[offer_chain: bigint, offer: bigint, market: bigint, offer_category: bigint, offer_company: bigint, offervalue: double, offer_brand: bigint, offer_dept: bigint, total_spend: double, day_of_week: double, day_of_month: double, month: double, has_bought_company: double, has_bought_company_q: double, has_bought_company_a: double, has_bought_company_30: double, has_bought_company_q_30: double, has_bought_company_a_30: double, has_bought_company_60: double, has_bought_company_q_60: double, has_bought_company_a_60: double, has_bought_company_90: double, has_bought_company_q_90: double, has_bought_company_a_90: double, has_bought_company_180: double, has_bought_company_q_180: double, has_bought_company_a_180: double, has_bought_category: double, has_bought_category_q: double, has_bought_category_a: double, has_bought_category_30: double, has_bought_category_q_30: double, has_bought_category_a_30: double, has_bought_category_60: double, has_bought_category_q_60: double, has_bought_cate

In [75]:
cvModel = crossval.fit(train_df)

In [76]:
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(cvModel.bestModel.stages[-1].coefficients))
print("Intercept: " + str(cvModel.bestModel.stages[-1].intercept))

Coefficients: [-0.00021268621136,8.52570830079e-05,-0.00360507936997,8.25132282822,0.000168524554689,1.29870264487e-09,-0.311250743646,1.73977822604e-06,0.0168459261006,3.71170153568e-06,-0.0148926841412,-0.00717350161858,0.909008971754,0.000766168517679,0.000192083813055,0.000140905817756,0.0513684788081,0.0084997328229,0.00681638103274,0.0189044703092,0.00314103892444,0.00264817686979,0.0105846511406,0.00166479252982,0.00160083901208,0.00308900513662,0.000547425180504,0.000575256998085,0.00710364117571,0.00121889197032,0.000423134280496,0.0682267049869,0.0222819797416,0.00458415850766,0.0295162770439,0.00688105577181,0.00131771371859,0.0205751093733,0.00543957958506,0.00113006462949,0.0120531768131,0.00317615797174,0.000682246586395,0.00523636672102,0.00136888058705,0.000577888954771,0.140064362997,0.0303953874848,0.0140175962607,0.0609542566711,0.0141912751971,0.00592389279545,0.0392390969293,0.00968538013675,0.00407642562136,0.0162148501202,0.00435397553599,0.00175411715114,0.04198

# Predicting on training data 

In [77]:
train_pred = cvModel.transform(train_df)

In [78]:
train_pred.select('probability', 'prediction').show()

+--------------------+----------+
|         probability|prediction|
+--------------------+----------+
|[0.99961210929115...|       0.0|
|[0.99984913889841...|       0.0|
|[0.36965916464421...|       1.0|
|[0.99718165945590...|       0.0|
|[0.99969608317672...|       0.0|
|[0.99986816816284...|       0.0|
|[0.99961446439226...|       0.0|
|[0.99960767338824...|       0.0|
|[0.99984992055939...|       0.0|
|[0.65505132003607...|       0.0|
|[0.99985809084751...|       0.0|
|[0.99985196054892...|       0.0|
|[0.99984768171038...|       0.0|
|[0.09065252490991...|       1.0|
|[4.29971472427468...|       1.0|
|[0.53250963237168...|       0.0|
|[0.50161725196770...|       0.0|
|[0.99986320385462...|       0.0|
|[7.17447161655919...|       1.0|
|[0.99985328741319...|       0.0|
+--------------------+----------+
only showing top 20 rows



# Predicting on test data

In [79]:
test_pred = cvModel.transform(test_df)

IllegalArgumentException: u'Field "label" does not exist.'

In [ ]:
test_pred.select('probability', 'prediction').show()

In [ ]:
test_id = test_df.select('id').toPandas()
test_prob = test_pred.select('probability').toPandas()

In [ ]:
# I didn't know how to access the vector inside the probability column.
# To work around this, I looped through the probability column and used the indexing operations to pull out
# the individual probabilities.
prob = []
for i in range(test_prob.shape[0]):
    prob.append(test_prob['probability'][i][1])
test_prob['repeatProbability'] = prob

In [ ]:
test_final_pred = pd.merge(test_id, test_prob, left_index=True, right_index=True)
test_final_pred.drop('probability', 1, inplace=True)

In [ ]:
test_final_pred.head()

In [ ]:
test_final_pred['repeatProbability'].describe()

In [ ]:
# Write the test file to csv
test_final_pred.to_csv('submission.csv', index=False)